In [1]:
import yaml
import os
from pathlib import Path
from string import Template
from cognite.client import CogniteClient, global_config
from cognite.client.data_classes.filters import SpaceFilter
from cognite.client.data_classes.data_modeling.instances import InvolvedContainers

file_path = Path("../pygen/cognite-sdk-config.yaml")

env_sub_template = Template(file_path.read_text())
file_env_parsed = env_sub_template.substitute(dict(os.environ))

cognite_config = yaml.safe_load(file_env_parsed)

global_config.apply_settings(cognite_config["global"])
client = CogniteClient.load(cognite_config["client"])


C:\Users\Filipe\AppData\Roaming\Python\Python312\site-packages\ipykernel\ipkernel.py:766: UserWarning: You are using version='7.74.1' of the SDK, however version='7.74.5' is available. To suppress this warning, either upgrade or do the following:
>>> from cognite.client.config import global_config
>>> global_config.disable_pypi_version_check = True
  _threading_Thread_run(self)


#### Containers

In [2]:
#list all containers from a specific space
space_name = "radix_space"
container_list = client.data_modeling.containers.list(space=space_name).to_pandas()
container_list


,space,external_id,properties,constraints,indexes,used_for,is_global,last_updated_time,created_time
0,radix_space,CopyOfActivity,{'copyOfActivityGUID': {'type': {'list': False...,{},{},all,False,2025-04-04 15:54:35.812,2025-04-04 15:54:35.812
1,radix_space,CopyOfAsset,"{'copyOfAssetGUID': {'type': {'list': False, '...",{},{},all,False,2025-04-04 15:54:35.812,2025-04-04 15:54:35.812
2,radix_space,CopyOfDescribable,{'copyOfDescribableGUID': {'type': {'list': Fa...,{},{},all,False,2025-04-04 15:54:35.812,2025-04-04 15:54:35.812
3,radix_space,CopyOfEquipment,{'copyOfEquipmentGUID': {'type': {'list': Fals...,{},{},all,False,2025-04-10 16:31:57.969,2025-04-10 16:31:57.969
4,radix_space,CopyOfTimeSeries,{'copyOfTimeSeriesGUID': {'type': {'list': Fal...,{},{},all,False,2025-04-04 15:54:35.812,2025-04-04 15:54:35.812
5,radix_space,Location,"{'latitude': {'type': {'list': False, 'unit': ...",{},{},all,False,2025-03-28 16:15:40.552,2025-03-28 16:15:40.552
6,radix_space,MetMast,"{'iecCompliant': {'type': {'list': False, 'typ...",{},{},all,False,2025-03-28 16:15:40.552,2025-03-28 16:15:40.552
7,radix_space,MonthlyLoss,"{'metric': {'type': {'list': False, 'collation...",{},{},all,False,2025-04-09 14:36:58.004,2025-04-09 14:36:58.004
8,radix_space,RadixHorizontalActivity,{'RadixHorizontalActivityGUID': {'type': {'lis...,{},{},all,False,2025-03-28 16:15:40.552,2025-03-28 16:15:40.552
9,radix_space,RadixHorizontalAsset,"{'location': {'type': {'list': False, 'type': ...",{},{},all,False,2025-03-28 16:15:40.552,2025-03-28 16:15:40.552


In [ ]:
#delete all containers from a specific space
for c in container_list.external_id:
    print(c)
    # client.data_modeling.containers.delete((space_name, c))
# client.data_modeling.containers.delete((space_name, "MonthlyLoss"))

CopyOfActivity
CopyOfAsset
CopyOfDescribable
CopyOfEquipment
CopyOfTimeSeries
Location
MetMast
MonthlyLoss
RadixHorizontalActivity
RadixHorizontalAsset


[ContainerId(space='radix_space', external_id='MonthlyLoss')]

#### Views

In [5]:
#list all view from a specific space
view_list = client.data_modeling.views.list(space=space_name).to_pandas()
# view_list

In [ ]:
#delete all views from a specific space
for v in view_list.external_id:
    print(v)
    # client.data_modeling.views.delete((space_name, v,"v1"))
# client.data_modeling.views.delete((space_name, "MonthlyLoss","v1"))

CopyOfActivity
CopyOfAsset
CopyOfDescribable
CopyOfEquipment
CopyOfTimeSeries
Location
MetMast
MonthlyLoss
RadixHorizontalActivity
RadixHorizontalAsset


[ViewId(space='radix_space', external_id='MonthlyLoss', version='v1')]

#### Data Models

In [14]:
client.data_modeling.data_models.list(space="radix_space").to_pandas()

,space,external_id,name,description,version,views,is_global,last_updated_time,created_time
0,radix_space,RadixEnterpriseDataModel,Radix Enterprise Data Model,Radix OEE model made to demonstrate process of...,v1,"[{'space': 'cdf_cdm', 'external_id': 'Cognite3...",False,2025-04-10 18:58:14.432,2025-04-10 18:58:14.432


In [ ]:
# client.data_modeling.data_models.delete(("radix_space", "OEEDataModel","v1"))

[DataModelId(space='radix_oee_space', external_id='OEEDataModel', version='v1')]

In [ ]:
dry_run = False
filter = SpaceFilter(space_name)
deleted_total = 0
for batch in client.data_modeling.instances(instance_type="node", filter=filter, chunk_size=1000):
    result = client.data_modeling.instances.inspect(
        nodes=batch.as_ids(),
        involved_containers=InvolvedContainers()
    )
    nodes_without_container_data = [
        (node.space, node.external_id) for node in result.nodes
        if not node.inspection_results.involved_containers
    ]
    if dry_run:
        for space, external_id in nodes_without_container_data:
            print(f"{space}:{external_id}")
    else:
        client.data_modeling.instances.delete(nodes=nodes_without_container_data)
        deleted_total += len(nodes_without_container_data)
        print(f"Deleted {len(nodes_without_container_data)} nodes without container data")

print(f"Deleted a total of {deleted_total} nodes")

In [ ]:
client.data_modeling.spaces.delete(space_name)